### Importing packages

In [1]:
import pandas as pd
import numpy as np
from shapely.wkt import loads as load_wkt
import geopandas as gpd

In [9]:
def generate_location_area_dim(zipcodes):
    """
    Generate a location area dimension DataFrame from the given zipcodes DataFrame.

    Args:
        zipcodes (DataFrame): A DataFrame containing ZIP codes and their geometries.

    Returns:
        DataFrame: A DataFrame containing the location area dimension data.
    """
    # Convert 'the_geom' to geometry and calculate centroids
    zipcodes['geometry'] = zipcodes['the_geom'].apply(load_wkt)
    gdf = gpd.GeoDataFrame(zipcodes, geometry='geometry')
    gdf['centroid'] = gdf['geometry'].centroid
    gdf['centroid_latitude'] = gdf['centroid'].y
    gdf['centroid_longitude'] = gdf['centroid'].x

    # Generate LocationAreaKey by concatenating latitude and longitude without special characters
    gdf['LocationAreaKey'] = (
        gdf['centroid_longitude'].astype(str).str.replace('.', '', regex=False).str.replace('-', '', regex=False) +
        gdf['centroid_latitude'].astype(str).str.replace('.', '', regex=False).str.replace('-', '', regex=False)
    )

    # Prepare the location area dimension DataFrame
    location_area_dim = pd.DataFrame({
        'LocationAreaKey': gdf['LocationAreaKey'],
        'Zipcode': gdf['ZIPCODE'],
        'MailCity': gdf['MAIL_CITY'],
        'ShapeLength': gdf['Shape_Leng'],
        'ShapeArea': gdf['Shape_Area'],
        'CeontroidLatitude': gdf['centroid_latitude'],
        'CeontroidLongitude': gdf['centroid_longitude']
    })

    return location_area_dim


In [10]:
test = generate_location_area_dim(zipcodes)

In [11]:
test

,LocationAreaKey,Zipcode,MailCity,ShapeLength,ShapeArea,CeontroidLatitude,CeontroidLongitude
0,771429374349402238966626952792325,20812,GLEN ECHO,14504.506513,9.810937e+06,38.966627,-77.142937
1,77118062490465383895647982277882,20816,BETHESDA,58716.124845,1.175655e+08,38.956480,-77.118062
2,77161105321519638973139992753595,20818,CABIN JOHN,31103.235156,3.599683e+07,38.973140,-77.161105
3,770964436175735838994223955611645,20894,BETHESDA,1298.401522,2.547457e+04,38.994224,-77.096444
4,77096473083286753899526140460659,20892,BETHESDA,140.742936,7.448763e+02,38.995261,-77.096473
...,...,...,...,...,...,...,...
92,77326517857141883918827696373,20841,BOYDS,250533.236842,7.723518e+08,39.188277,-77.326518
93,77232719485357753920877090545198,20876,GERMANTOWN,135940.688675,3.258196e+08,39.208771,-77.232719
94,77056853884695713920369643184604,20833,BROOKEVILLE,207931.092330,5.585343e+08,39.203696,-77.056854
95,774362176657559839190007380164744,20842,DICKERSON,357054.654503,1.131929e+09,39.190007,-77.436218
